# Imports

In [8]:
from IPython.display import display as dsp
from datetime import datetime as dt
from random import sample


import sys
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', 400)

import canalyst_candas.candas as cd
from canalyst_candas.configuration.config import Config

config_info = {
  "canalyst_api_key": "",
  "s3_access_key_id": "",
  "s3_secret_key": "",
  "fred_key": "",
  "default_dir": "",
  "mds_host": "",
  "wp_host": ""
}

config = Config(config=config_info)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



# Creating A Search Object

In [2]:
# Instantiate new search object:
search = cd.Search(config)


# Searching for Consumer Companies with Stores

In [4]:
# A search for "Consumer" companies with exposure to "store":
search_results_df = search.search_time_series(
    sector="Consumer",
    time_series_name="store"
)

Building search index...
Done


In [9]:
# There are over 1m rows in the search index:
dsp(search.df_search.shape)
dsp(search.df_search.head())

(1273274, 27)

,Unnamed: 0,Bloomberg Ticker,Path,Filename,CSIN,index,ticker,period_name,period_duration_type,period_start_date,...,unit_description,unit_symbol,unit_type,value,is_historical,category_slug,MRQ,is_driver,name_index,period_name_sorted
0,0,GOOGL US,US Companies/Communication Services/Interactive Media & Services,Alphabet GOOGL US.xlsx,U07N2S0124,21876,GOOGL US,Q3-2021,fiscal_quarter,2021-07-01,...,Percentage,%,percentage,44.00,True,NaN,Q3-2021,True,7.00,2021Q3
1,1,GOOGL US,US Companies/Communication Services/Interactive Media & Services,Alphabet GOOGL US.xlsx,U07N2S0124,22161,GOOGL US,Q3-2021,fiscal_quarter,2021-07-01,...,Percentage,%,percentage,43.04,True,NaN,Q3-2021,True,8.00,2021Q3
2,2,GOOGL US,US Companies/Communication Services/Interactive Media & Services,Alphabet GOOGL US.xlsx,U07N2S0124,16632,GOOGL US,Q3-2021,fiscal_quarter,2021-07-01,...,Percentage,%,percentage,43.84,True,NaN,Q3-2021,False,9.00,2021Q3
3,3,GOOGL US,US Companies/Communication Services/Interactive Media & Services,Alphabet GOOGL US.xlsx,U07N2S0124,18627,GOOGL US,Q3-2021,fiscal_quarter,2021-07-01,...,Percentage,%,percentage,39.84,True,NaN,Q3-2021,True,10.00,2021Q3
4,4,GOOGL US,US Companies/Communication Services/Interactive Media & Services,Alphabet GOOGL US.xlsx,U07N2S0124,24669,GOOGL US,Q3-2021,fiscal_quarter,2021-07-01,...,Percentage,%,percentage,43.23,True,NaN,Q3-2021,False,11.00,2021Q3


In [10]:
# Common time series:
gpd_search = search_results_df.groupby("time_series_name")[["ticker"]].nunique().reset_index().copy()
gpd_search.sort_values(by="ticker", ascending=False, inplace=True)
gpd_search.head(10)

,time_series_name,ticker
128,MO_OS_Stores_EoP,112
125,MO_OS_Stores_Closed,97
131,MO_OS_Stores_New,96
122,MO_OS_Stores_BoP,96
22,MO_OS_AverageStores,79
105,MO_OS_SSS_Stores,71
67,MO_OS_SSS_NewStores,69
81,MO_OS_SSS_SalesPerCompStore,64
75,MO_OS_SSS_SalesFromCompStores,63
78,MO_OS_SSS_SalesFromNewStores,63


In [11]:
# Common time series drivers:
driver_df = search_results_df[search_results_df["is_driver"] == True].copy()
gpd_search = driver_df.groupby("time_series_name")[["ticker"]].nunique().copy()
gpd_search.sort_values(by="ticker", ascending=False, inplace=True)
gpd_search.reset_index(inplace=True)
dsp(gpd_search)

# Turn into a list:
most_common_driver = gpd_search["time_series_name"].iloc[0]
dsp(driver_df[driver_df["time_series_name"].str.contains(most_common_driver)].head(10))
dsp(driver_df[driver_df["time_series_name"].str.contains(most_common_driver)].shape)
dsp(driver_df[driver_df["time_series_name"].str.contains(most_common_driver)]["ticker"].nunique())
driver_df[driver_df["time_series_name"].str.contains(most_common_driver)].to_clipboard()

,time_series_name,ticker
0,MO_OS_Stores_New,60
1,MO_OS_Stores_Closed,60
2,MO_OS_SSS_NewStoreEfficiency,54
3,MO_OS_Stores_New_CompanyOwned,16
4,MO_OS_Stores_Closed_CompanyOwned,15
...,...,...
828,z_9ZBMO80150_MO_OS_Canadastoresclosed,1
829,z_9ZBMO80150_MO_OS_Canadastoresopened,1
830,z_9ZBMO80150_MO_OS_DSWNewStoreEfficiency,1
831,z_9ZBMO80150_MO_OS_DSWstoresclosed,1


,ticker,CSIN,Path,Filename,category,time_series_description,time_series_name,period_duration_type,unit_type,is_driver,value,search_matches
725454,ZUMZ US,8X9RM80141,"US Companies/Consumer Discretionary/Apparel, Accessories & Luxury/Apparel Retail",Zumiez ZUMZ US.xlsx,Operating Stats - Store Analysis (FS),"Stores opened, # of stores",MO_OS_Stores_New,fiscal_quarter,count,True,3.00,1
577115,WMK US,WIF7CE0146,US Companies/Consumer Staples/Food & Beverage/Hypermarkets & Super Centers,Weis Markets WMK US.xlsx,Operating Stats - Revenue per Retail Store (FS),"Additions of supermarket stores, # of stores",MO_OS_Stores_New,fiscal_quarter,count,True,0.00,1
74896,WING US,HUWCDD0172,US Companies/Consumer Discretionary/Restaurants/Casual Dining,Wingstop WING US.xlsx,Operating Stats - Company-Owned Analysis,"Domestic company-owned openings during period, # of restaurants",MO_OS_Stores_New_CompanyOwned,fiscal_quarter,count,True,1.00,1
554482,VRA US,MBCNOK0167,"US Companies/Consumer Discretionary/Apparel, Accessories & Luxury/Accessories",Vera Bradley VRA US.xlsx,Operating Stats - VB Direct Store Analysis (MD&A),"VB Direct Store opened, # of stores",MO_OS_Stores_New,fiscal_quarter,count,True,3.00,1
85569,ULTA US,EYUO400147,US Companies/Consumer Staples/Personal Products/Beauty Retail,Ulta Beauty ULTA US.xlsx,Operating Stats - Same Store Analysis,"Store opened, # of stores",MO_OS_Stores_New,fiscal_quarter,count,True,7.00,1
713080,TLYS US,O6EOCO0152,"US Companies/Consumer Discretionary/Apparel, Accessories & Luxury/Apparel Retail",Tillys TLYS US.xlsx,Operating Stats - Retail Stores SSS (PR),"Opened stores, # of stores",MO_OS_Stores_New,fiscal_quarter,count,True,6.00,1
64715,TGT US,A1RO3K0126,US Companies/Consumer Discretionary/Staples Retail/General Merchandise Stores,Target TGT US.xlsx,Operating Stats - Store Data,"Stores Opened, # of stores",MO_OS_Stores_New,fiscal_quarter,count,True,2.00,1
568395,TCS US,K0FA000179,US Companies/Consumer Discretionary/Specialty Stores,Container Store TCS US.xlsx,Operating Stats - TCS Store Analysis (MD&A),"Openings, # of stores",MO_OS_Stores_New,fiscal_quarter,count,True,0.00,1
744217,TACO US,N44XEU0149,US Companies/Consumer Discretionary/Restaurants/Quick Service Restaurant,Del Taco TACO US.xlsx,Operating Stats - Store Franchised Stores (PR),"Franchised restaurants additions, # of restaurants",MO_OS_Stores_New_Franchised,fiscal_quarter,count,True,3.00,1
744182,TACO US,N44XEU0149,US Companies/Consumer Discretionary/Restaurants/Quick Service Restaurant,Del Taco TACO US.xlsx,Operating Stats - Store Company-operated Stores (PR),"Company-operated restaurants opened, # of restaurants",MO_OS_Stores_New_CompanyOwned,fiscal_quarter,count,True,1.00,1


(87, 12)

77

In [12]:
# A list of tickers for this driver:
ts = gpd_search["time_series_name"].iloc[0]
store_tickers = list(driver_df[driver_df["time_series_name"] == ts]["ticker"].unique())
len(store_tickers)

60

# Create a Model Set of "Store Companies"

In [14]:
# Create a model set form this ticker list:
start = dt.now()
store_set = cd.ModelSet(ticker_list=store_tickers[0:4], config=config, allow_nulls=True)

# Tkaes around 5 minutes:
dsp((dt.now() - start).seconds)

10

In [15]:
start = dt.now()
# Create params:
df_params = store_set.forecast_frame(ts,
                             n_periods=range(0,4),
                             function_name='multiply',
                             function_value=1.1)
# Tkaes around 5 minutes:
dsp((dt.now() - start).seconds)
dsp(df_params.shape)
dsp(df_params.head())

0

(16, 6)

,ticker,period_name,time_series_name,value,new_value,period_end_date
636,ULTA US,Q3-2021,MO_OS_Stores_New,10.00,11.00,2021-10-30
2484,VRA US,Q3-2022,MO_OS_Stores_New,2.00,2.20,2021-10-30
5656,ZUMZ US,Q3-2021,MO_OS_Stores_New,11.00,12.10,2021-10-30
4292,WMK US,Q4-2021,MO_OS_Stores_New,1.00,1.10,2022-01-01
637,ULTA US,Q4-2021,MO_OS_Stores_New,10.00,11.00,2022-01-29


In [19]:
# Pick a sample of just 4:
sample_tickers = sample(list(df_params["ticker"].unique()), 4)
sample_params = df_params[df_params["ticker"].isin(sample_tickers)].copy()
sample_params.shape

(16, 6)

# Run Scenarios on Common Time Series

In [20]:
start = dt.now()

# How much does revenue move?
return_series = "MO_RIS_REV"

# Scenarios for a sample of 10:
dicts_output=store_set.fit(sample_params,return_series)

# Print time:
dsp((dt.now() - start).seconds)

VRA US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/MBCNOK0167/equity-models/Q2-2022.25/scenarios/uQNew4G0UkmGs7ujZXgD6g/
ZUMZ US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/EYUO400147/equity-models/Q2-2021.20/scenarios/Bs8ZJ2I9X76pcOBqr7bKFg/
WMK US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/WIF7CE0146/equity-models/Q3-2021.22/scenarios/qXZdr4ZwXtGsWpKj-XyiIQ/
ULTA US scenario_id_url: https://mds.canalyst.com/api/equity-model-series/EYUO400147/equity-models/Q2-2021.20/scenarios/Bs8ZJ2I9X76pcOBqr7bKFg/


22

In [22]:
# Show scenario results:
for k in dicts_output.keys():
    dsp(dicts_output[k])

,ticker,period_name,time_series_name,time_series_description,default,scenario,diff
0,VRA US,Q3-2022,MO_RIS_REV,Net Revenue,139182978.27880,139277616.078,1.00068
1,VRA US,Q4-2022,MO_RIS_REV,Net Revenue,162909232.50469,163138349.341,1.00141
2,VRA US,Q1-2023,MO_RIS_REV,Net Revenue,119715061.13654,119964496.3692,1.00208
3,VRA US,Q2-2023,MO_RIS_REV,Net Revenue,152827048.49435,153282482.943,1.00298
4,VRA US,Q3-2023,MO_RIS_REV,Net Revenue,143882456.95239,144277659.2222,1.00275
5,VRA US,Q4-2023,MO_RIS_REV,Net Revenue,168824296.35373,169295989.0952,1.00279


,ticker,period_name,time_series_name,time_series_description,default,scenario,diff
0,ZUMZ US,Q3-2021,MO_RIS_REV,Net Revenue,298066315.32429,1656504440.615,5.55750
1,ZUMZ US,Q4-2021,MO_RIS_REV,Net Revenue,348585942.20036,2343567993.6454,6.72307
2,ZUMZ US,Q1-2022,MO_RIS_REV,Net Revenue,239776171.09798,2054032312.6595,8.56646
3,ZUMZ US,Q2-2022,MO_RIS_REV,Net Revenue,274153717.65139,2060193658.7309,7.51474
4,ZUMZ US,Q3-2022,MO_RIS_REV,Net Revenue,302658327.94702,1792487817.6027,5.92248
5,ZUMZ US,Q4-2022,MO_RIS_REV,Net Revenue,352499702.25677,2557885780.8133,7.25642


,ticker,period_name,time_series_name,time_series_description,default,scenario,diff
0,WMK US,Q4-2021,MO_RIS_REV,Net Revenue,1121752561.05363,1122066946.457,1.00028
1,WMK US,Q1-2022,MO_RIS_REV,Net Revenue,1029631418.04974,1029946933.2953,1.00031
2,WMK US,Q2-2022,MO_RIS_REV,Net Revenue,1065246174.03327,1065571274.595,1.00031
3,WMK US,Q3-2022,MO_RIS_REV,Net Revenue,1084842026.86126,1085173107.8461,1.00031
4,WMK US,Q4-2022,MO_RIS_REV,Net Revenue,1044464154.52257,1045135184.4062,1.00064


,ticker,period_name,time_series_name,time_series_description,default,scenario,diff
0,ULTA US,Q3-2021,MO_RIS_REV,Net Revenue,1655248489.39422,1656504440.615,1.00076
1,ULTA US,Q4-2021,MO_RIS_REV,Net Revenue,2340020561.59407,2343567993.6454,1.00152
2,ULTA US,Q1-2022,MO_RIS_REV,Net Revenue,2049998158.25329,2054032312.6595,1.00197
3,ULTA US,Q2-2022,MO_RIS_REV,Net Revenue,2054200529.81889,2060193658.7309,1.00292
4,ULTA US,Q3-2022,MO_RIS_REV,Net Revenue,1786882011.44315,1792487817.6027,1.00314
5,ULTA US,Q4-2022,MO_RIS_REV,Net Revenue,2549313836.95305,2557885780.8133,1.00336
